# [Based on this guide](https://barrowclift.me/post/wireguard-server-on-macos)

# UPDATES
- Servers running macOS will not have a .app file. 
- Must only be run from python source (for using `sudo python rmcc_server.py`) 
- Will have a manual toggle for the Wireguard post-up and post-down commands, along with onscreen walkthrough of when and how to enable/disable from the Wireguard GUI

# Postup
```shell
#!/bin/sh

# 1) Preparing the directory which we'll persist the pf token
#    generated by (2) in. That token can then be used by our
#    postdown.sh script to remove the routing rule in the
#    event Wireguard is shut down on the server.
mkdir -p /usr/local/var/run/wireguard
chmod 700 /usr/local/var/run/wireguard

# 2) Dynamically add the NAT rule, enable the firewall, increase
#    its reference count (-E) and persist the reference token
#    generated by the command into pf_wireguard_token.txt,
#    which postdown.sh will reference when Wireguard is shut
#    down.
echo 'nat on en0 from 10.0.10.0/24 to any -> (en0)' | \
    pfctl -a com.apple/wireguard -Ef - 2>&1 | \
    grep 'Token' | \
    sed 's%Token : \(.*\)%\1%' > /usr/local/var/run/wireguard/pf_wireguard_token.txt   
```

In [4]:
print("nat on en0 from 10.0.10.0/24 to any -> (en0)")

# must be run with sudo:
# pfctl -a com.apple/wireguard -Ef - 2>&1
# -a : anchor the app with signature com.apple/wireguard
# -Ef : enable packet filter firewall
# 2>&1 : redirect stderr to stdout (print any errors)

# grep "Token"

test


# Postdown
```shell
#!/bin/sh

# 1) Fetch the pf reference token that was generated on
#    Wireguard startup with postup.sh
TOKEN=`cat /usr/local/var/run/wireguard/pf_wireguard_token.txt`

# 2) Remove the reference (and by extension, the pf rule that
#    generated it). Adding and removing rules by references
#    like this will automatically disable the packet filter
#    firewall if there are no other references left, but will
#    leave it up and intact if there are.
pfctl -X ${TOKEN} || exit 1
rm -f /usr/local/var/run/wireguard/pf_wireguard_token.txt
```

# Make executable
```shell
chmod u+x /usr/local/etc/wireguard/*.sh
```

# Install Wireguard tools
```shell
brew install wireguard-tools
```

User must:
- Install homebrew: https://brew.sh
- Either:
    - Install cakebrew: https://www.cakebrew.com and then install wireguard-tools via the cakebrew
    - Or just `brew install wireguardtools`

# Configure wireguard
```shell
umask 077 # Ensure credentials don't leak from possible race condition.
wg genkey | tee privatekey | wg pubkey > publickey
```

# Create Server configurations
```python
[Interface]
# Substitute with the subnet you chose for Wireguard earlier.
Address = 10.0.10.0/24
# Substitute with your *server's* private key
PrivateKey = XXX
# If you chose a different port earlier when setting up port
# forwarding on your router, update the port here to match.
ListenPort = 51820
# This prevents IPv4 & IPv6 DNS leaks when browsing the web on the
# VPN. I chose Cloudflare's public DNS servers, but feel free to use
# whatever provider you prefer.
DNS = 1.1.1.1, 1.0.0.1, 2606:4700:4700::1111
# This ensures our peers continue to report their Wireguard-
# assigned IPs while connected to the VPN. This is required for
# their traffic to get routed correctly by the firewall rules we
# crafted earlier with pf.
PostUp = /usr/sbin/sysctl -w net.inet.ip.forwarding=1
PostUp = /usr/sbin/sysctl -w net.inet6.ip6.forwarding=1
# Adds the firewall routing rule on Wireguard server startup
PostUp = /usr/local/etc/wireguard/postup.sh
# Removes the firewall routing rule on Wireguard server shutdown
PostDown = /usr/local/etc/wireguard/postdown.sh

[Peer]
# Substitute with *this peer*'s public key
PublicKey = XXX
# Chose a unique IP within the Wireguard subnet you defined earlier
# that this particular peer will use when connected to the VPN.
AllowedIPs = 10.0.10.10/32

# Follow the same steps as the [Peer] template above for each
# additional peer you wish to connect to the VPN with.
```

In [ ]:
# Include all of the above **except** the postup.sh and postdown.sh commands... We'll run those here, manually, as defined above.

# Create Client configurations
```python
[Interface]
# This MUST match the "AllowedIPs" IP you assigned to this peer in
# the server's config.
Address = 10.0.10.10/32
# Substitute with *this peer's* private key.
PrivateKey = XXX
# This prevents IPv4 & IPv6 DNS leaks when browsing the web on the
# VPN. I chose Cloudflare's public DNS servers, but feel free to use
# whatever provider you prefer.
DNS = 1.1.1.1, 1.0.0.1, 2606:4700:4700::1111

[Peer]
# Substitute with your *server's* public key
PublicKey = XXX
# Your Wireguard server's public IP. If you chose a different port
# earlier when setting up port forwarding on your router, update the
# port here to match.
Endpoint = XXX.XXX.XXX.XXX:51820
# Informs Wireguard to forward ALL traffic through the VPN.
AllowedIPs = 0.0.0.0/0, ::/0
# If you're be behind a NAT, this will keep the connection alive.
PersistentKeepalive = 25
```

# Setup service
```xml
<?xml version="1.0" encoding="UTF-8"?>
<!DOCTYPE plist PUBLIC "-//Apple//DTD PLIST 1.0//EN" "http://www.apple.com/DTDs/PropertyList-1.0.dtd">
<plist version="1.0">
    <dict>
        <key>Label</key>
        <string>com.wireguard.server</string>
        <key>ProgramArguments</key>
        <array>
            <string>/usr/local/bin/wg-quick</string>
            <string>up</string>
            <string>/usr/local/etc/wireguard/coordinates.conf</string>
        </array>
        <key>KeepAlive</key>
        <true/>
        <key>RunAtLoad</key>
        <true/>
        <key>StandardErrorPath</key>
        <string>/usr/local/var/log/wireguard.err</string>
        <key>EnvironmentVariables</key>
        <dict>
            <key>PATH</key>
            <string>/usr/local/sbin:/usr/local/bin:/usr/bin:/bin:/usr/sbin:/sbin</string>
        </dict>
    </dict>
</plist>
```
## Inform launchd of the service
```shell
sudo launchctl enable system/com.wireguard.server
sudo launchctl bootstrap system /Library/LaunchDaemons/com.wireguard.server.plist
```